Fitting the best linear predictor BLP for the JPTA data to test for heterogeneity.

First, we need models for the baseline functions that 

In [41]:
# imports
import pandas as pd
from econml.grf import CausalForest
from econml.grf import CausalIVForest

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# loading the data
jtpa = pd.read_csv("jtpa_doubleclean.csv")
jtpa.head(3)

,recid,radate,assignmt,site,training,afdc,sex,class_tr,ojt_jsa,oth_serv,...,hsorged,black,hispanic,age2225,age2629,age3035,age3644,age4554,f2sms,wkless13
0,300001,05/16/89,1,NE,1,0,0,0,0,1,...,1.0,1,0,0,0,0,0,1,0,1.0
1,300002,08/30/89,1,LC,1,0,0,0,0,1,...,1.0,0,0,1,0,0,0,0,0,0.0
2,300006,08/18/88,1,HF,0,0,0,1,0,0,...,1.0,0,1,0,0,1,0,0,0,1.0


In [24]:
# split data into auxiliary and main sample, stratified by assignment
auxiliary_index, main_index = train_test_split(jtpa.index, stratify=jtpa['assignmt'], test_size=0.7, random_state=42)

auxiliary = jtpa.loc[auxiliary_index].reset_index()
main = jtpa.drop(auxiliary_index).reset_index()
print(len(auxiliary), len(main))

2922 6821


In [25]:
# how many in auxiliary are untreated to learn baseline?
auxiliary[auxiliary["assignmt"] == 0].shape

(968, 27)

In [26]:
# define X
features = ['afdc', 'sex', 'married', 'pbhous', 'hsorged', 'black', 'hispanic', 'wkless13','age', 'prevearn']

In [43]:
# fit random forest to learn E(Y|D=0, X)
X_aux = auxiliary[features + ["assignmt"] + ["training"]]
Y_aux = auxiliary["earnings"]

# get grid for regularization
grid = {"n_estimators": [10, 50, 100], "max_depth": [2, 5, 8], "min_samples_leaf": [4, 6, 10]}
rf = RandomForestRegressor()
gs = GridSearchCV(rf, grid, cv = 4, verbose=1)
gs.fit(X = X_aux[X_aux["assignmt"] == 0][features], y = Y_aux[X_aux["assignmt"] == 0])

# get the best model
b = gs.best_estimator_

Fitting 4 folds for each of 27 candidates, totalling 108 fits


In [28]:
b.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 2,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 6,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 50,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [44]:
# fit causal forest to learn E(Y(0) - Y(1)|X)

cf = CausalForest(min_samples_leaf=2)
cf.fit(X = X_aux[features], T = X_aux["assignmt"], y = Y_aux)

cf_iv = CausalIVForest(min_samples_leaf=2)
cf_iv.fit(X = X_aux[features], T = X_aux["training"], y = Y_aux, Z = X_aux["assignmt"])

CausalIVForest(min_samples_leaf=2)

In [45]:
# get predictions on main sample
b_preds = b.predict(main[features])
cf_preds = pd.Series(cf.predict_full(main[features])[:,0]) # extract from (n,1) array
cf_iv_preds = pd.Series(cf_iv.predict_full(main[features])[:,0]) # extract from (n,1) array


We can now fit the BLP for $s_0(X)$. This this outcome is unobserved, Chernozukov et al. suggests to use a certain weighted linear projection, where the weights are given by the inverse of the variance of the propensity score. Since in our random experiment, the propensity score is constant, i.e. $e(X) = p \: \forall X$, the weights are also constant and can be ignored.

In [51]:
# build Weighted Residual BLP to estimate coefficients of BLP
# Strategy A in Chernozhukov et al. (2018)

p = main.assignmt.mean()

# get dependent variables
finite_sample_improv = pd.DataFrame({"const" : 1, "bZ" : b_preds})
intercept = pd.Series(main.assignmt - p, name = "intercept")
slope = pd.Series((main.assignmt - p)*(cf_iv_preds - cf_iv_preds.mean()), name = "slope")

# collect all dependent variables
dependent_vars = pd.concat([finite_sample_improv, intercept, slope], axis = 1)

In [52]:
dependent_vars.sample(6)

,const,bZ,intercept,slope
2632,1,12421.038476,0.331183,3993.056994
1020,1,22043.437620,0.331183,712.059439
6672,1,16406.922277,0.331183,5001.257911
5041,1,11073.071286,-0.668817,-3035.139168
2353,1,22840.281695,0.331183,833.603442
2588,1,12352.591504,-0.668817,-3611.179959


In [53]:
# run linear model
model = sm.OLS(main.earnings, dependent_vars)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               earnings   R-squared:                       0.115
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     295.1
Date:                Tue, 23 Apr 2024   Prob (F-statistic):          4.00e-180
Time:                        10:06:05   Log-Likelihood:                -75647.
No. Observations:                6821   AIC:                         1.513e+05
Df Residuals:                    6817   BIC:                         1.513e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        531.7385    561.851      0.946      0.344    -569.665    1633.142
bZ             1.0323      0.035     29.574      0.000       0.964       1.101
intercept   1452.2179    408.080      3.559      0.000     652.254    2252.182
slope         -0.0077      0.086     -0.090      0.929      -0.176       0.161
==============================================================================
Omnibus:                     1745.148   Durbin-Watson:                   2.027
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4854.034
Skew:                           1.350   Prob(JB):                         0.00
Kurtosis:                       6.128   Cond. No.                     4.71e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.71e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

This is clearly not significant, which is surprising...


In [12]:
# also using a standard RF to predict E(Y|D=1, X)

# fit random forest to learn E(Y|D=1, X) using parameters from b

# fit random forest to learn E(Y|D=0, X)
# get grid for regularization
rf2 = RandomForestRegressor(**b.get_params())
rf2.fit(X = X_aux[X_aux["assignmt"] == 1][features], y = Y_aux[X_aux["assignmt"] == 1])

# get alternative cate_pred
b_preds2 = rf2.predict(main[features])
cate_pred = pd.Series(b_preds2 - b_preds)

In [13]:
# replace slope with cate_pred
dependent_vars["slope"] = cate_pred

In [14]:
# rename and re run
dependent_vars.rename({"bZ": "B(x)", "intercept": "\alpha", "slope": "[S(x)-ES(x)]"}, inplace = True, axis = 1)

# re run linear model
model = sm.OLS(main.earnings, dependent_vars)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               earnings   R-squared:                       0.144
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     327.3
Date:                Tue, 23 Apr 2024   Prob (F-statistic):          1.96e-196
Time:                        09:49:31   Log-Likelihood:                -64814.
No. Observations:                5846   AIC:                         1.296e+05
Df Residuals:                    5842   BIC:                         1.297e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1027.4284    586.494     -1.752      0.080   -2177.173     122.316
B(x)             1.0576      0.034     30.984      0.000       0.991       1.125
lpha         1198.9679    440.936      2.719      0.007     334.570    2063.366
[S(x)-ES(x)]     0.6467      0.067      9.641      0.000       0.515       0.778
==============================================================================
Omnibus:                     1450.475   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4043.910
Skew:                           1.306   Prob(JB):                         0.00
Kurtosis:                       6.126   Cond. No.                     4.74e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.74e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
latex_BLP = results.summary().tables[1].as_latex_tabular()

In [16]:
print(latex_BLP)

\begin{center}
\begin{tabular}{lcccccc}
\toprule
                      & \textbf{coef} & \textbf{std err} & \textbf{t} & \textbf{P$> |$t$|$} & \textbf{[0.025} & \textbf{0.975]}  \\
\midrule
\textbf{const}        &   -1027.4284  &      586.494     &    -1.752  &         0.080        &    -2177.173    &      122.316     \\
\textbf{B(x)}         &       1.0576  &        0.034     &    30.984  &         0.000        &        0.991    &        1.125     \\
\textbf{lpha}        &    1198.9679  &      440.936     &     2.719  &         0.007        &      334.570    &     2063.366     \\
\textbf{[S(x)-ES(x)]} &       0.6467  &        0.067     &     9.641  &         0.000        &        0.515    &        0.778     \\
\bottomrule
\end{tabular}
\end{center}
